# Requirements

In [5]:
from collections import Counter
from functools import reduce, partial
import random

# Applying function to iterables

Consider the following lists as running examples.

In [1]:
l1 = list(range(10))
l2 = random.choices(range(3), k=7)
print(l1, l2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [1, 2, 2, 2, 0, 0, 0]


## map

The `map` function can be used to apply a function to any element of an iterable, e.g.,

In [2]:
list(map(str, map(lambda x: x % 2 == 0, l1)))

['True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False',
 'True',
 'False']

Note that the result of `map` is not a list, but rather an iterator, even when it is appllied to a list.

## filter

The `filter` function applies a Boolean predicate to each element of an iterable, and retains only those elements for which the predicate evaluates to `True`.  In the example below, only the even elements will be returned.

In [3]:
list(filter(lambda x: x % 2 == 0, l1))

[0, 2, 4, 6, 8]

## reduce

Python has quite number of reduction functions such as `sum`, `max`, and `min`, however, using the `reduce` function in `functools` we can easily write our own.

In [5]:
def incr(c, x):
    c[x] += 1
    return c
reduce(incr, l2, Counter())

Counter({1: 1, 2: 3, 0: 3})

However, note that there is no performance advantage in using `reduce` for this particular case:

In [6]:
l_long = random.choices(range(5), k=1_000_000)

In [7]:
%timeit reduce(incr, l_long, Counter())

260 ms ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
counter = Counter()
for x in l_long:
    counter[x] += 1

283 ms ± 73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Higher order functions

A new function can be created out of an existing function using `partial` to fix values for one or more arguments of the original function.

In [10]:
def line_function(x, slope, intercept):
    return slope*x + intercept

In [11]:
line1 = partial(line_function, slope=2.0, intercept=3.0)

In [12]:
line1(0.0), line1(1.0)

(3.0, 5.0)

More generally, we can define a closure, i.e., a function that returns another function.

In [13]:
def create_line(slope, intercept):
    def line(x):
        return slope*x + intercept
    return line

In [14]:
line2 = create_line(2.0, 3.0)

In [15]:
line2(0.0), line2(1.0)

(3.0, 5.0)

Closures can also be used to accumulate data.  The following closure will generate a function that returns a value that is incremented by one on each invocation.

In [10]:
def create_counter():
    n = 0
    def counter():
        nonlocal n
        n += 1
        return n
    return counter

In [11]:
c1, c2 = create_counter(), create_counter()

In [12]:
for _ in range(3):
    print(c1())
for _ in range(3):
    print(c1(), c2())

1
2
3
4 1
5 2
6 3


`nonlocal` ensures that the `n` in outer scope is used, i.e., the variable `n` defined in the generating function `create_counter`.

# Coroutines

Coroutines are special functions that have at least one, but possibly multiple entry points for suspending and resuming execution, while retaining state between suspend and resume.

In the function below, the function's execution will be suspended at the `yield` expression, and will be resumed when the funciton is called with the number of the next floor for the elevator. This is accomplished using the `send` method defined on coroutine functions.

In [24]:
def create_elevator(highest_floor):
    def elevator():
        prev_floor = None
        curr_floor = 0
        while True:
            next_floor = (yield f'current floor: {curr_floor}' if prev_floor is None else f'move from {prev_floor} to {curr_floor}')
            if next_floor is not None:
                if -1 <= next_floor <= highest_floor:
                    prev_floor = curr_floor
                    curr_floor = next_floor
                else:
                    print(f'no such floor: {next_floor}')
            else:
                break
    my_elevator = elevator()
    print(next(my_elevator))
    return my_elevator    

In [25]:
my_elevator = create_elevator(3)

current floor: 0


In [26]:
for floor in random.choices([-1, 0, 1, 2, 3], k=10):
    print(f'press {floor}: {my_elevator.send(floor)}')

press 0: move from 0 to 0
press -1: move from 0 to -1
press 1: move from -1 to 1
press 3: move from 1 to 3
press 3: move from 3 to 3
press 1: move from 3 to 1
press 0: move from 1 to 0
press 3: move from 0 to 3
press -1: move from 3 to -1
press 0: move from -1 to 0
